# Коллаборативная фильтрация

Коллаборативная фильтрация основана на идее, что пользователей, похожих на меня, можно использовать для прогнозирования того, насколько мне понравится тот или иной продукт или услуга, которые эти пользователи испытали на себе, а я - нет.

In [42]:
import pandas as pd
import numpy as np
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
from surprise import KNNBasic
from surprise import accuracy
from surprise import SlopeOne

from surprise.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
from sklearn.model_selection import GridSearchCV

In [22]:
reader = Reader()
ratings = pd.read_csv('ratings_small.csv')
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


Буду тестировать разные модели из surprise - библиотека, предназначенная для построения и анализа рекомендательных систем:
* Загружаем датасет.
* Проводим кросс-валидацию. Кросс-валидация — это метод оценки производительности модели на разных подмножествах данных, чтобы оценить насколько хорошо она будет работать на новых данных. Будем использовать 5-кратную кросс-валидацию: данные будут разделены на 5 частей, и модель будет обучена на 4 из них и протестирована на оставшейся 5-й, и это будет повторено 5 раз, чтобы все 5 частей послужили тестовыми.
* Обучаем модель.
* Предсказываем.

## SVD

SVD - это метод линейной алгебры, который используется для разложения матрицы на три другие матрицы. В нашем случае, SVD используется для уменьшения размерности матрицы пользователь-фильм.

Как SVD работает в рекомендательных системах:
1.  Матрица пользователь-фильм: Исходные данные представляются в виде матрицы, где строки — пользователи, столбцы — фильмы, и ячейки содержат известные рейтинги.
2.  Сингулярное разложение: Матрица разлагается на три:
    * Матрица пользователей: Представляет пользователей в векторном пространстве. Каждый ряд соответствует пользователю, а столбцы - факторам, которые могут влиять на предпочтения пользователя (например, фактор "любит драму", "любит комедии").
    * Диагональная матрица сингулярных значений: Содержит сингулярные значения, которые указывают на важность каждого фактора. Значения упорядочены по убыванию.
    * Транспонированная матрица фильмов: Представляет фильмы в векторном пространстве. Каждый столбец соответствует фильму, а строки - факторам, аналогичным факторам из матрицы пользователей.
3.  Уменьшение размерности: Оставляют только самые важные факторы (сингулярные значения) из матрицы сингулярных значений, тем самым сокращая размерность матриц пользователей и фильмов.
4.  Восстановление матрицы: Используя урезанные матрицы, можно восстановить (приблизить) исходную матрицу рейтингов.
5.  Предсказание: Предсказанные рейтинги используются для рекомендации фильмов пользователям.

In [44]:
svd = SVD()

cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

trainset = data.build_full_trainset()

prediction = svd.predict(1, 302, 3)
print(f"SVD Prediction for user 1 and movie 302: {prediction.est:.2f}")

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9022  0.8965  0.8987  0.8891  0.9011  0.8975  0.0046  
MAE (testset)     0.6917  0.6931  0.6948  0.6831  0.6956  0.6917  0.0045  
Fit time          7.30    8.94    7.07    8.05    8.66    8.01    0.73    
Test time         1.31    2.68    1.38    1.66    1.14    1.63    0.55    
SVD Prediction for user 1 and movie 302: 2.81


## KNNBasic

KNNBasic — это метод, который делает предсказания на основе рейтингов K пользователей, наиболее похожих на текущего пользователя. Он относится к группе алгоритмов "на основе памяти", потому что модель хранит все данные (чаще всего, после преобразования в матрицу схожести) и делает вычисления на основе этих данных, в отличие от алгоритмов "на основе модели", которые строят модель из данных (SVD).

Как KNNBasic работает:

1. Расчет схожести:
  * Вычисляется схожесть между всеми парами пользователей.
  * Используется pearson_baseline (в моем коде). Это модифицированный коэффициент корреляции Пирсона, который учитывает базовые рейтинги пользователей при расчете схожести, что обычно повышает точность. Базовый рейтинг пользователя - это средний рейтинг, который он ставит всем фильмам.
2. Поиск K ближайших соседей:
  * Для текущего пользователя выбираются K пользователей, которые имеют наибольшую схожесть.
3. Предсказание рейтинга:
  * Рейтинг для текущего пользователя и фильма предсказывается на основе рейтингов его K ближайших соседей.

In [35]:
sim_options = {'name': 'pearson_baseline', 'user_based': True}
knn = KNNBasic(sim_options=sim_options)

cross_validate(knn, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

trainset = data.build_full_trainset()

prediction = knn.predict(1, 302, 3)
print(f"KNN Prediction for user 1 and movie 302: {prediction.est:.2f}")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0022  1.0032  0.9948  0.9937  0.9946  0.9977  0.0041  
MAE (testset)     0.7736  0.7754  0.7679  0.7644  0.7706  0.7704  0.0039  
Fit time          0.74    0.68    0.71    0.69    0.68    0.70    0.02    
Test time         1.16    1.14    1.26    1.12    1.04  

## SlopeOne

SlopeOne — это алгоритм рекомендательных систем, основанный на сравнении разностей рейтингов между парами предметов (фильмов).

Как работает SlopeOne:

1. Расчет разностей рейтингов:
    * Для каждой пары фильмов вычисляется средняя разность рейтингов, которые поставили им пользователи.
    * Эти разности сохраняются в матрицу разностей.
2. Предсказание рейтинга:
    * Чтобы предсказать рейтинг пользователя для фильма, алгоритм берет рейтинги, которые этот пользователь поставил другим фильмам, и добавляет к ним средние разности между этими фильмами и фильмом, который нужно предсказать.
    * Предсказанный рейтинг является средним из всех этих "скорректированных" рейтингов.
    * Если есть несколько фильмов, которые пользователь оценил, то аналогичные вычисления проводятся для всех этих фильмов, и затем результаты усредняются.

In [36]:
slope_one = SlopeOne()
cross_validate(slope_one, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

trainset = data.build_full_trainset()

prediction = slope_one.predict(1, 302, 3)
print(f"SlopeOne Prediction for user 1 and movie 302: {prediction.est:.2f}")

Evaluating RMSE, MAE of algorithm SlopeOne on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9229  0.9341  0.9313  0.9316  0.9252  0.9290  0.0042  
MAE (testset)     0.7079  0.7145  0.7128  0.7136  0.7092  0.7116  0.0026  
Fit time          2.46    3.37    2.58    2.64    2.54    2.72    0.33    
Test time         4.74    4.52    4.34    4.52    4.49    4.52    0.13    
SlopeOne Prediction for user 1 and movie 302: 2.54


### Подбор гиперпараметров для SVD

In [23]:
param_grid = {
    'n_factors': [50, 100, 150],
    'reg_all': [0.02, 0.1, 0.5]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)
gs.fit(data)

print("Best RMSE:", gs.best_score['rmse'])
print("Best MAE:", gs.best_score['mae'])
print("Best parameters:", gs.best_params['rmse'])

best_svd_rmse = gs.best_estimator['rmse']

trainset = data.build_full_trainset()
best_svd_rmse.fit(trainset)

prediction = best_svd_rmse.predict(1, 302, 2.5)
print(f"Prediction: {prediction.est:.2f}")

Best RMSE: 0.8910287090640056
Best MAE: 0.687439718534794
Best parameters: {'n_factors': 50, 'reg_all': 0.1}
Prediction: 2.73


* n_factors (количество факторов): Это количество *скрытых факторов*, которые SVD использует для представления пользователей и фильмов.
* reg_all (регуляризация всех параметров): Это параметр, который контролирует *регуляризацию*, которая применяется ко всем параметрам модели.
Регуляризация — это техника, которая помогает предотвратить *переобучение* модели. Она как бы "штрафует" модель за слишком большие или сложные параметры, заставляя её находить более простые и обобщающие решения.
* lr_all (скорость обучения): Это параметр, который контролирует, насколько сильно параметры модели обновляются на каждой итерации обучения. Алгоритм обучения SVD постепенно меняет параметры модели, чтобы сделать предсказания более точными. lr_all определяет, на сколько сильно параметры обновляются на каждой итерации.
* n_epochs (количество эпох): Это количество раз, которое алгоритм SVD проходит через все тренировочные данные в процессе обучения.
Что это такое: Каждая эпоха — это один проход по всему набору данных для обучения модели.

In [32]:
def train_and_evaluate(data, n_iter=10):
    
    param_distributions = {
        'n_factors': randint(20, 200),
        'reg_all': uniform(0.01, 0.5),
        'lr_all': uniform(0.002, 0.02),
        'n_epochs': randint(10, 50)
    }

    rs = RandomizedSearchCV(SVD, param_distributions, measures=['rmse', 'mae'], cv=5, n_iter=n_iter)
    rs.fit(data)

    print("Best RMSE:", rs.best_score['rmse'])
    print("Best MAE:", rs.best_score['mae'])
    print("Best parameters (RMSE):", rs.best_params['rmse'])
    print("Best parameters (MAE):", rs.best_params['mae'])

    best_rmse = rs.best_score['rmse']
    best_mae = rs.best_score['mae']
    best_params_rmse = rs.best_params['rmse']
    best_params_mae = rs.best_params['mae']
    best_svd_rmse = rs.best_estimator['rmse']
    best_svd_mae = rs.best_estimator['mae']

    trainset = data.build_full_trainset()
    best_svd_rmse.fit(trainset)
    best_svd_mae.fit(trainset)

    return {
        'best_rmse': best_rmse,
        'best_mae': best_mae,
        'best_params_rmse': best_params_rmse,
        'best_params_mae': best_params_mae,
        'model_rmse': best_svd_rmse,
        'model_mae': best_svd_mae
    }


num_runs = 5
results = []

# Запускаем обучение и оценку несколько раз
for i in range(num_runs):
    print(f"Run {i+1}/{num_runs}")
    result = train_and_evaluate(data, n_iter=10)
    results.append(result)


Run 1/5
Best RMSE: 0.8845886068824523
Best MAE: 0.6823396790926728
Best parameters (RMSE): {'lr_all': 0.0152356581331288, 'n_epochs': 20, 'n_factors': 120, 'reg_all': 0.13516352219691535}
Best parameters (MAE): {'lr_all': 0.0152356581331288, 'n_epochs': 20, 'n_factors': 120, 'reg_all': 0.13516352219691535}
Run 2/5
Best RMSE: 0.8825978135804469
Best MAE: 0.6791322398987357
Best parameters (RMSE): {'lr_all': 0.014524748165272116, 'n_epochs': 13, 'n_factors': 179, 'reg_all': 0.05750356368218917}
Best parameters (MAE): {'lr_all': 0.014524748165272116, 'n_epochs': 13, 'n_factors': 179, 'reg_all': 0.05750356368218917}
Run 3/5
Best RMSE: 0.8729471580300328
Best MAE: 0.6708102330982632
Best parameters (RMSE): {'lr_all': 0.012953500641312315, 'n_epochs': 44, 'n_factors': 91, 'reg_all': 0.09408867005221243}
Best parameters (MAE): {'lr_all': 0.012953500641312315, 'n_epochs': 44, 'n_factors': 91, 'reg_all': 0.09408867005221243}
Run 4/5
Best RMSE: 0.8785716626025932
Best MAE: 0.6751491243999539
Bes

In [38]:
# Находим лучшие результаты среди всех запусков
best_rmse_run = min(results, key=lambda x: x['best_rmse'])
best_mae_run = min(results, key=lambda x: x['best_mae'])

# Выводим результаты
print("\nBest Results Overall:")
print(f"Best RMSE: {best_rmse_run['best_rmse']:.4f}")
print(f"Best parameters (RMSE): {best_rmse_run['best_params_rmse']}")
print(f"Best MAE: {best_mae_run['best_mae']:.4f}")
print(f"Best parameters (MAE): {best_mae_run['best_params_mae']}")

# Выводим предсказания
prediction_rmse = best_rmse_run['model_rmse'].predict(1, 302, 2.5)
prediction_mae = best_mae_run['model_mae'].predict(1, 302, 2.5)

print(f"Prediction (Best RMSE model): {prediction_rmse.est:.2f}")
print(f"Prediction (Best MAE model): {prediction_mae.est:.2f}")


Best Results Overall:
Best RMSE: 0.8729
Best parameters (RMSE): {'lr_all': 0.012953500641312315, 'n_epochs': 44, 'n_factors': 91, 'reg_all': 0.09408867005221243}
Best MAE: 0.6708
Best parameters (MAE): {'lr_all': 0.012953500641312315, 'n_epochs': 44, 'n_factors': 91, 'reg_all': 0.09408867005221243}
Prediction (Best RMSE model): 2.76
Prediction (Best MAE model): 2.83


### Итоговая модель:


In [45]:
best_params_rmse = {'lr_all': 0.012953500641312315, 'n_epochs': 44, 'n_factors': 91, 'reg_all': 0.09408867005221243}
best_params_mae = {'lr_all': 0.012953500641312315, 'n_epochs': 44, 'n_factors': 91, 'reg_all': 0.09408867005221243}

# Инициализация SVD с лучшими параметрами (RMSE)
svd = SVD(
    lr_all=best_params_rmse['lr_all'],
    n_epochs=best_params_rmse['n_epochs'],
    n_factors=best_params_rmse['n_factors'],
    reg_all=best_params_rmse['reg_all']
)

cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

trainset = data.build_full_trainset()

prediction = svd.predict(1, 302, 3)
print(f"SVD Prediction for user 1 and movie 302: {prediction.est:.2f}")

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8695  0.8800  0.8774  0.8730  0.8725  0.8745  0.0037  
MAE (testset)     0.6676  0.6752  0.6747  0.6674  0.6727  0.6715  0.0034  
Fit time          16.34   17.81   15.82   15.81   15.70   16.30   0.79    
Test time         2.96    1.34    1.22    1.44    1.14    1.62    0.68    
SVD Prediction for user 1 and movie 302: 2.88


### Топ 30 фильмов для пользователя

In [125]:
def get_top_n_recommendations(model, user_id, data, n=30):
    all_movie_ids = data.df['movieId'].unique()
    rated_movie_ids = data.df[data.df['userId'] == user_id]['movieId'].unique()

    unrated_movie_ids = np.setdiff1d(all_movie_ids, rated_movie_ids)

    predictions = [model.predict(user_id, movie_id) for movie_id in unrated_movie_ids]

    predictions.sort(key=lambda x: x.est, reverse=True)

    top_n_movie_ids = [pred.iid for pred in predictions[:n]]
    return top_n_movie_ids


In [126]:
user_id_to_recommend = 1

top_30_movies = get_top_n_recommendations(best_svd_rmse, user_id_to_recommend, data, n=30)
print(f"Top 30 movies for user {user_id_to_recommend}: {top_30_movies}")

Top 30 movies for user 1: [858, 318, 913, 7502, 969, 926, 1221, 2064, 3462, 5971, 904, 50, 899, 905, 78499, 1252, 106782, 1254, 26729, 1276, 2917, 2019, 1060, 3730, 307, 2300, 1228, 246, 1203, 1945]
